# Comparison of the snia_toy_06 model #

In [3]:
# uncomment the following line to download the code comparison stuff
#!wget http://opensupernova.org/~wkerzend/files/ccompare1_models.tgz
#!tar zxvf ccompare1_models.tgz

x ddc10/._eden_ddc10_artis.txt
x ddc10/eden_ddc10_artis.txt
x ddc10/eden_ddc10_cmfgen.txt
x ddc10/._edep_ddc10_artis.txt
x ddc10/edep_ddc10_artis.txt
x ddc10/edep_ddc10_cmfgen.txt
x ddc10/._edep_ddc10_sedonalinetransfer.txt
x ddc10/edep_ddc10_sedonalinetransfer.txt
x ddc10/._edep_ddc10_supernu.txt
x ddc10/edep_ddc10_supernu.txt
x ddc10/._ionfrac_al_ddc10_artis.txt
x ddc10/ionfrac_al_ddc10_artis.txt
x ddc10/._ionfrac_ar_ddc10_artis.txt
x ddc10/ionfrac_ar_ddc10_artis.txt
x ddc10/._ionfrac_c_ddc10_artis.txt
x ddc10/ionfrac_c_ddc10_artis.txt
x ddc10/._ionfrac_ca_ddc10_artis.txt
x ddc10/ionfrac_ca_ddc10_artis.txt
x ddc10/._ionfrac_cl_ddc10_artis.txt
x ddc10/ionfrac_cl_ddc10_artis.txt
x ddc10/._ionfrac_co_ddc10_artis.txt
x ddc10/ionfrac_co_ddc10_artis.txt
x ddc10/._ionfrac_cr_ddc10_artis.txt
x ddc10/ionfrac_cr_ddc10_artis.txt
x ddc10/._ionfrac_cu_ddc10_artis.txt
x ddc10/ionfrac_cu_ddc10_artis.txt
x ddc10/._ionfrac_f_ddc10_artis.txt
x ddc10/ionfrac_f_ddc10_artis.txt
x ddc10/._ionfrac_fe_ddc10

x toy06/._spectra_toy06_sedonalinetransfer.txt
x toy06/spectra_toy06_sedonalinetransfer.txt
x toy06/._spectra_toy06_sumo.txt
x toy06/spectra_toy06_sumo.txt
x toy06/._spectra_toy06_supernu.txt
x toy06/spectra_toy06_supernu.txt
x toy06/._spectra_toy06_urilight.txt
x toy06/spectra_toy06_urilight.txt
x toy06/._tgas_toy06_artis.txt
x toy06/tgas_toy06_artis.txt
x toy06/tgas_toy06_cmfgen.txt
x toy06/._tgas_toy06_sumo.txt
x toy06/tgas_toy06_sumo.txt
x toy06/._tgas_toy06_supernu.txt
x toy06/tgas_toy06_supernu.txt


In [8]:
import os, itertools
import pandas as pd

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5 as palette
from glob import glob

output_notebook()

# create a color iterator
colors = itertools.cycle(palette)


# create a new plot with a title and axis labels
p = figure(title="Comparison of Lbol", x_axis_label='x', y_axis_label='y')

for fname, color in zip(glob('toy06/lbol_edep_toy06_*.txt'), colors):
    lbol = pd.read_csv(fname, delim_whitespace=True, comment='#')
    lbol.columns = ['time', 'lbol', 'edep']
# add a line renderer with legend and line thickness
    name = os.path.basename(fname).split('_')[-1][:-4]
    p.line(lbol.time, lbol.lbol, legend=name, line_width=2, color=color)
p.legend.location = "top_right"
p.legend.click_policy="hide"

# show the results
show(p)

Loading BokehJS ...

'sedona'